In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

HAGO ENDPINT CON FECHAS + FILTRADO

In [2]:
# Definir el rango de fechas que te interesa
fecha_inicio = "2025-03-01"
fecha_fin = "2025-03-02"
url = f"https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json?dtstart={fecha_inicio}&dtend={fecha_fin}"


In [3]:
# Realizar la solicitud
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Convertir las fechas de inicio y fin a objetos datetime
    fecha_inicio_dt = datetime.strptime(fecha_inicio, "%Y-%m-%d")   # ESTO NO LO ENTIENDO
    fecha_fin_dt = datetime.strptime(fecha_fin, "%Y-%m-%d")
    
    # Filtrar los datos para asegurarse de que solo incluyan eventos dentro del rango de fechas
    eventos_filtrados = []
    for evento in data['@graph']:
        fecha_evento = datetime.strptime(evento['dtstart'], "%Y-%m-%d %H:%M:%S.%f")
        if fecha_inicio_dt <= fecha_evento <= fecha_fin_dt:
            eventos_filtrados.append(evento)
    
    # Mostrar los datos filtrados
    print(eventos_filtrados)
else:
    print("Error al obtener los datos:", response.status_code)

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/evento/12682299-asalto-lectura-tres-cerditos-mas.json', '@type': 'https://datos.madrid.es/egob/kos/actividades/RecitalesPresentacionesActosLiterarios', 'id': '12682299', 'title': 'Asalto a la lectura. Tres cerditos y más', 'description': '', 'free': 1, 'price': '', 'dtstart': '2025-03-01 11:00:00.0', 'dtend': '2025-03-01 23:59:00.0', 'time': '11:00', 'excluded-days': '', 'audience': 'Niños,Familias', 'uid': '12682299', 'link': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=6aec3e3332fe4910VgnVCM1000001d4a900aRCRD', 'event-location': 'Centro Cultural Clara del Rey - Museo ABC (Centro)', 'references': {'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ad0d88f2de54910VgnVCM1000001d4a900aRCRD'}, 'relation': {'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/11876732-centro-cultural-clara-rey-museo

In [4]:
data_dic = eventos_filtrados

In [5]:
eventos = []

for evento in data_dic:
    id_evento = evento.get("id", None)
    nombre_evento = evento.get("title", "")
    url_evento = evento.get("link", "")
    horario = evento.get("time", None)
    organizacion = evento.get("organization", {}).get("organization-name", None)

    # 🔍 1. Intentar obtener la dirección desde "event-location"
    direccion = evento.get("event-location", None)

    # 🔍 2. Si no hay dirección, buscar en "description"
    if not direccion:
        descripcion = evento.get("description", "")
        if "Lugar:" in descripcion:  # ✅ Verifica si la descripción contiene "Lugar:"
            direccion = descripcion.split("Lugar:")[1].strip()  # Extrae la parte después de "Lugar:"
    
    # 🔍 3. Obtener código postal
    direccion_info = evento.get("address", {}).get("area", {})
    codigo_postal = direccion_info.get("postal-code", None)

    # Agregar a la lista de eventos
    eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, evento.get("dtstart", ""), evento.get("dtend", ""), organizacion))

# Convertir a DataFrame
df_api = pd.DataFrame(eventos, columns=["id_evento", "nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "fecha_inicio", "fecha_fin", "organizacion"])


In [6]:
shape =df_api.shape
print(shape)
df_api.sample(10)

(45, 9)


,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
2,12682147,Baile de Carnaval. Concierto de Origen,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,plaza de Puerto Rubio (antigua plaza Vieja). C...,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,None
12,12690609,Cuatro historias de alquiler,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,Centro Cultural Buero Vallejo (San Blas - Cani...,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Buero Vallejo (San Blas - Cani...
18,12683859,Fiesta de carnaval en el Barro de la Alameda d...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,18:00,2025-03-01 18:00:00.0,2025-03-01 23:59:00.0,None
40,12693997,Tejiendo Colores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,,2025-03-01 00:00:00.0,2025-03-31 23:59:00.0,Centro de Educación Ambiental y Cultural Maris...
4,12697705,By Lucipun ' Muladar creativo',http://www.madrid.es/sites/v/index.jsp?vgnextc...,28001,"Espacio Cultural Núñez de Balboa, 40 (Salamanca)",,2025-03-01 00:00:00.0,2025-03-21 23:59:00.0,"Espacio Cultural Núñez de Balboa, 40 (Salamanca)"
9,12697798,Clara Torres Morenilla,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28028,Centro Cultural Quinta del Berro - Rafael Alta...,,2025-03-01 00:00:00.0,2025-03-15 23:59:00.0,Centro Cultural Quinta del Berro - Rafael Alta...
19,12233828,Itinerario Ornitológico Parque Emperatriz Marí...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,09:00,2025-03-01 09:00:00.0,2025-03-01 23:59:00.0,None
44,12686173,Villaverde Bajo. Concurso de Disfraces y Pasac...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,17:30,2025-03-01 17:30:00.0,2025-03-01 23:59:00.0,None
23,12697883,Le kiosque,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Cineteca Madrid,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Cineteca Madrid
5,12694234,Carnaval en familia. Espectáculo familiar. Car...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28042,Centro Cultural - Centro Socio Cultural Villa ...,12:30,2025-03-01 12:30:00.0,2025-03-01 23:59:00.0,Centro Cultural - Centro Socio Cultural Villa ...


In [7]:
def completar_horario(df):
    """
    Extrae la hora de la columna 'fecha_inicio' y la usa para llenar 'horario' si está vacío.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'fecha_inicio' y 'horario'.
        
    Retorna:
        pd.DataFrame: DataFrame con 'horario' corregido.
    """
    # Extraer la hora de 'fecha_inicio' (primeros 16 caracteres para evitar milisegundos)
    df["hora_extraida"] = df["fecha_inicio"].astype(str).str[11:16]  # Formato HH:MM
    
    # Si 'horario' está vacío, completar con 'hora_extraida'
    df["horario"] = df["horario"].mask(df["horario"] == "", df["hora_extraida"])
    
    # Eliminar la columna auxiliar
    df.drop(columns=["hora_extraida"], inplace=True)
    
    return df


In [8]:
completar_horario(df_api)
df_api.sample(10)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
33,12690327,Senderismo urbano. Emilia Pardo Bazán: mujer e...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,Centro de Información y Educación Ambiental de...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro de Información y Educación Ambiental de...
14,12697300,El mago Rafathor,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28035,Centro Cultural Julio Cortázar (Moncloa - Arav...,12:00,2025-03-01 12:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Julio Cortázar (Moncloa - Arav...
2,12682147,Baile de Carnaval. Concierto de Origen,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,plaza de Puerto Rubio (antigua plaza Vieja). C...,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,None
39,12686923,Talleres Infantiles de Carnaval. Centros Socio...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,None
37,12695827,Taller 'Pequeñas científicas',http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,Aula ambiental La Cabaña del Retiro,16:30,2025-03-01 16:30:00.0,2025-03-01 23:59:00.0,Aula ambiental La Cabaña del Retiro
23,12697883,Le kiosque,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Cineteca Madrid,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Cineteca Madrid
29,12661729,Nosotras creamos mundo,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28041,Centro Sociocultural San Fermín (Usera),12:00,2025-03-01 12:00:00.0,2025-03-01 23:59:00.0,Centro Sociocultural San Fermín (Usera)
16,12694176,Exposición de Pintura `Paisajes´ y `Rincones d...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28021,Centro Sociocultural Marconi (Villaverde),00:00,2025-03-01 00:00:00.0,2025-03-31 23:59:00.0,Centro Sociocultural Marconi (Villaverde)
38,12696013,Talleres de máscaras,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28020,Centro Sociocultural Tetuán,00:00,2025-03-01 00:00:00.0,2025-03-01 23:59:00.0,Centro Sociocultural Tetuán
40,12693997,Tejiendo Colores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,00:00,2025-03-01 00:00:00.0,2025-03-31 23:59:00.0,Centro de Educación Ambiental y Cultural Maris...


In [9]:
df_api["fecha_inicio"].value_counts().sort_index()

fecha_inicio
2025-03-01 00:00:00.0    9
2025-03-01 08:30:00.0    1
2025-03-01 09:00:00.0    1
2025-03-01 10:00:00.0    1
2025-03-01 10:30:00.0    2
2025-03-01 11:00:00.0    8
2025-03-01 12:00:00.0    3
2025-03-01 12:30:00.0    1
2025-03-01 16:30:00.0    1
2025-03-01 17:00:00.0    1
2025-03-01 17:30:00.0    1
2025-03-01 18:00:00.0    4
2025-03-01 19:00:00.0    5
2025-03-01 19:30:00.0    3
2025-03-01 21:15:00.0    1
2025-03-01 21:30:00.0    3
Name: count, dtype: int64

In [10]:
celdas_vacias = df_api.isnull().sum() + (df_api == '').sum()
celdas_vacias

id_evento         0
nombre_evento     0
url_evento        0
codigo_postal    15
direccion        11
horario           0
fecha_inicio      0
fecha_fin         0
organizacion     14
dtype: int64

## Normalización

In [11]:
def normalizar_fechas(df):
    """
    Elimina las horas de las columnas 'fecha_inicio' y 'fecha_fin', dejando solo 'YYYY-MM-DD'.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'fecha_inicio' y 'fecha_fin'.
        
    Retorna:
        pd.DataFrame: DataFrame con las fechas normalizadas.
    """
    df["fecha_inicio"] = df["fecha_inicio"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    df["fecha_fin"] = df["fecha_fin"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    
    return df


In [12]:
df_api = normalizar_fechas(df_api)
df_api.sample(3)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
35,12696746,Taller de Bancal lasaña en el Huerto Urbano Co...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,11:00,2025-03-01,2025-03-01,None
3,12682149,Baile de Carnaval. Concierto de Versioneta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,plaza de Puerto Rubio (antigua plaza Vieja). V...,21:15,2025-03-01,2025-03-01,None
23,12697883,Le kiosque,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Cineteca Madrid,19:00,2025-03-01,2025-03-01,Cineteca Madrid


## Carga de datos

## Paso 1. Conectar con Postgres

In [13]:
import psycopg2
from psycopg2 import OperationalError

def create_connection():
    try:
        # Configurar la conexión
        conn = psycopg2.connect(
            dbname="project_etl_ibis",
            user="postgres",
            password="admin",
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa")
        return conn
    except OperationalError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Crear la conexión
conn = create_connection()

if conn:
    # Crear un cursor para ejecutar comandos SQL
    cur = conn.cursor()

    # Verificar la conexión
    cur.execute("SELECT version();")
    print(cur.fetchone())

    # Cerrar la conexión
    cur.close()
    conn.close()

Conexión exitosa
('PostgreSQL 17.0 on x86_64-windows, compiled by msvc-19.41.34120, 64-bit',)


## 2. Carga de Datos

### Inserto datos en TABLA CIUDAD

In [14]:
import psycopg2

# Conectar a la base de datos
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Obtener el nombre de la ciudad desde el DataFrame (suponiendo que todas son "Madrid")
nombre_ciudad = "Madrid"

# 1️⃣ Verificar si la ciudad ya existe
cur.execute("SELECT id_ciudad FROM ciudad WHERE nombre_ciudad = %s", (nombre_ciudad,))
result = cur.fetchone()

if result:
    id_ciudad = result[0]  # Si ya existe, obtenemos su ID
    print(f"La ciudad '{nombre_ciudad}' ya existe con id_ciudad = {id_ciudad}")
else:
    # 2️⃣ Si no existe, insertamos la ciudad y obtenemos el id_ciudad
    cur.execute("INSERT INTO ciudad (nombre_ciudad) VALUES (%s) RETURNING id_ciudad;", (nombre_ciudad,))
    id_ciudad = cur.fetchone()[0]
    conn.commit()
    print(f"Se insertó la ciudad '{nombre_ciudad}' con id_ciudad = {id_ciudad}")

# Cerrar conexión
cur.close()
conn.close()


La ciudad 'Madrid' ya existe con id_ciudad = 1


QUERY

In [15]:
# Conectarse a PostgreSQL
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Obtener todas las ventas
cur.execute("SELECT * FROM ciudad;")

# Mostrar los resultados
for row in cur.fetchall():
    print(row)

# Cerrar conexión
cur.close()
conn.close()

(1, 'Madrid')


### Cargar datos en TABLA HOTELES

In [16]:
df_completo = pd.read_csv(r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\dfcompleto.csv", parse_dates=["fecha_reserva", "inicio_estancia", "final_estancia"])

In [17]:
df_completo.rename(columns={"valoración": "valoracion"}, inplace=True)  # Renombrar la columna
df_completo.dtypes

id_reserva                 object
id_cliente                  int64
nombre                     object
apellido                   object
mail                       object
competencia                  bool
fecha_reserva      datetime64[ns]
inicio_estancia    datetime64[ns]
final_estancia     datetime64[ns]
id_hotel                    int64
precio_noche              float64
nombre_hotel               object
valoracion                float64
ciudad                     object
dtype: object

In [18]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# 1️⃣ Obtener la ciudad del DataFrame (asumiendo que todas son "Madrid")
nombre_ciudad = df_completo["ciudad"].unique()[0]  # Si hay más ciudades, necesitarás un bucle

# 2️⃣ Verificar si la ciudad ya existe
cur.execute("SELECT id_ciudad FROM ciudad WHERE nombre_ciudad = %s", (nombre_ciudad,))
result = cur.fetchone()

if result:
    id_ciudad = result[0]  # Recuperar el ID de la ciudad
    print(f"La ciudad '{nombre_ciudad}' ya existe con id_ciudad = {id_ciudad}")
else:
    # Insertar la ciudad si no existe
    cur.execute("INSERT INTO ciudad (nombre_ciudad) VALUES (%s) RETURNING id_ciudad;", (nombre_ciudad,))
    id_ciudad = cur.fetchone()[0]
    conn.commit()
    print(f"Se insertó la ciudad '{nombre_ciudad}' con id_ciudad = {id_ciudad}")

# 3️⃣ Preparar los datos de hoteles
df_hoteles = df_completo[["nombre_hotel", "competencia", "valoracion"]].copy()
df_hoteles["id_ciudad"] = id_ciudad  # Asignar el id_ciudad obtenido
df_hoteles["competencia"] = df_hoteles["competencia"].astype(bool)

# 4️⃣ Eliminar duplicados para evitar insertar el mismo hotel varias veces
df_hoteles = df_hoteles.drop_duplicates(subset=["nombre_hotel"])

# 5️⃣ Convertir DataFrame a lista de tuplas para la inserción
records = df_hoteles[["nombre_hotel", "competencia", "valoracion", "id_ciudad"]].to_records(index=False)
records_list = [(str(nombre_hotel), bool(competencia), float(valoracion), int(id_ciudad)) for nombre_hotel, competencia, valoracion, id_ciudad in records]

# 6️⃣ Insertar los datos en la tabla `hoteles`
query = """
    INSERT INTO hoteles (nombre_hotel, competencia, valoracion, id_ciudad) 
    VALUES (%s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Hoteles insertados correctamente.")

La ciudad 'Madrid' ya existe con id_ciudad = 1
Hoteles insertados correctamente.


## Tabla clientes 

In [19]:
df_completo.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'valoracion', 'ciudad'],
      dtype='object')

In [20]:
df_cliente = df_completo[["id_cliente", "nombre", "apellido", "mail"]].copy()

In [21]:
# Convertir id_cliente a string antes de insertarlo
df_cliente["id_cliente"] = df_cliente["id_cliente"].astype(str)


In [22]:
df_cliente.dtypes

id_cliente    object
nombre        object
apellido      object
mail          object
dtype: object

In [23]:
df_cliente = df_cliente.drop_duplicates(subset=["id_cliente"])

In [24]:
# Reabrir la conexión
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas
records = df_cliente[["id_cliente", "nombre", "apellido", "mail"]].to_records(index=False)
records_list = list(records)

# Insertar múltiples registros con ejecuta_many()
query = """
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")

UniqueViolation: llave duplicada viola restricción de unicidad «clientes_pkey»
DETAIL:  Ya existe la llave (id_cliente)=(2).


In [25]:
conn.rollback()

rollback

In [ ]:
df_cliente["id_cliente"].count()

14905

QUERY

SELECT r.id_reserva, c.id_cliente
FROM reservas r
LEFT JOIN clientes c ON r.id_cliente = c.id_cliente
WHERE c.id_cliente IS NULL;  -- Detecta reservas sin cliente


TABLA RESERVAS

In [26]:
df_completo.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'valoracion', 'ciudad'],
      dtype='object')

In [27]:
df_reservas = df_completo[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel"]].copy()

In [28]:
df_reservas.dtypes

id_reserva                 object
fecha_reserva      datetime64[ns]
inicio_estancia    datetime64[ns]
final_estancia     datetime64[ns]
precio_noche              float64
id_cliente                  int64
id_hotel                    int64
dtype: object

In [29]:
df_reservas["fecha_reserva"] = pd.to_datetime(df_reservas["fecha_reserva"], errors='coerce')
df_reservas["inicio_estancia"] = pd.to_datetime(df_reservas["inicio_estancia"], errors='coerce')
df_reservas["final_estancia"] = pd.to_datetime(df_reservas["final_estancia"], errors='coerce')


In [30]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()


el problema q me daba es q la fecha reserva tenia q crear cuando extraje los datos una columna poniendo la fecha del estract

In [31]:
from datetime import datetime, timedelta
fecha_ayer = pd.to_datetime(datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

# Reemplazar los valores NaT en la columna 'fecha_reserva' con la fecha de ayer
df_reservas['fecha_reserva'] = df_reservas['fecha_reserva'].fillna(fecha_ayer)

completar datos inicio estancia y fin estancia 
precio noche


In [33]:
# Definir la fecha con la que se rellenarán los valores nulos
fecha_reemplazo = '2025-03-01'

# Rellenar los valores nulos en la columna 'inicio_estancia' con la fecha '2025-03-01'
df_reservas['inicio_estancia'] = df_reservas['inicio_estancia'].fillna(fecha_reemplazo)


In [34]:
# Calcular la moda de la columna 'final_estancia'
moda_final_estancia = df_reservas['final_estancia'].mode()[0]
print(moda_final_estancia)

2025-03-02 00:00:00


In [35]:
# Rellenar los valores nulos en la columna 'final_estancia' con la moda
df_reservas['final_estancia'] = df_reservas['final_estancia'].fillna(moda_final_estancia)

In [36]:
df_reservas['final_estancia'].value_counts()

final_estancia
2025-03-02    15000
Name: count, dtype: int64

In [37]:
# Calcular la mediana de la columna 'precio_noche'
mediana_precio_noche = df_reservas['precio_noche'].median()

# Rellenar los valores nulos en la columna 'precio_noche' con la mediana
df_reservas['precio_noche'] = df_reservas['precio_noche'].fillna(mediana_precio_noche)

In [ ]:
records_list = df_reservas.values.tolist()
records_list ## crear una lista d lsitas 

In [43]:
df_reservas['id_cliente'] = df_reservas['id_cliente'].astype(str)

usar for iter rows porq el records lo convierte en string y luego tienes q volver a transformar

In [45]:

#✅ Convertir fechas a string en formato "YYYY-MM-DD"
#df_reservas["fecha_reserva"] = df_reservas["fecha_reserva"].dt.strftime('%Y-%m-%d')
#df_reservas["inicio_estancia"] = df_reservas["inicio_estancia"].dt.strftime('%Y-%m-%d') ## por que si lo hice antes tengo q volver a pasarlo a datatime
#df_reservas["final_estancia"] = df_reservas["final_estancia"].dt.strftime('%Y-%m-%d')

# ✅ Asegurar que precio_noche es un float positivo

# Convertir DataFrame a lista de tuplas para la inserción


# Insertar en la tabla `reservas`
query = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)"""

cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Reservas insertadas correctamente.")


Reservas insertadas correctamente.


TABLA EVENTOS

In [48]:
df_api.dtypes

id_evento        object
nombre_evento    object
url_evento       object
codigo_postal    object
direccion        object
horario          object
fecha_inicio     object
fecha_fin        object
organizacion     object
dtype: object

In [58]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_evento      45 non-null     object        
 1   nombre_evento  45 non-null     object        
 2   url_evento     45 non-null     object        
 3   codigo_postal  45 non-null     object        
 4   direccion      45 non-null     object        
 5   horario        45 non-null     object        
 6   fecha_inicio   45 non-null     datetime64[ns]
 7   fecha_fin      45 non-null     datetime64[ns]
 8   organizacion   45 non-null     object        
dtypes: datetime64[ns](2), object(7)
memory usage: 3.3+ KB


In [52]:
df_api['codigo_postal'] = df_api['codigo_postal'].fillna("no se encuentra")
df_api['organizacion'] = df_api['organizacion'].fillna("no se encuentra")

In [57]:
df_api["fecha_inicio"] = pd.to_datetime(df_api['fecha_inicio'], errors='coerce')
df_api["fecha_fin"] = pd.to_datetime(df_api['fecha_fin'], errors='coerce')

ABRO CONEXION

In [ ]:
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

records_list = df_api.values.tolist()


# Insertar en la tabla `reservas`
query = """
    INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

cur.executemany(query, records_list)

# 📌 Guardar cambios y cerrar conexión
conn.commit()
cur.close()
conn.close()

print("Reservas insertadas correctamente.")

